# WonderCat Notebook

In [2]:
import requests, base64, warnings, re
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

## Call API and Store Data

In [3]:
%%time

"""
WordPress API Credentials and Functions
"""
api_prefix = 'https://env-1120817.us.reclaim.cloud/wp-json/wp/v2/user-experience'

def get_total_pagecount():
    api_url = f'{api_prefix}?page=1&per_page=100'
    response = requests.get(api_url)
    pages_count = response.headers['X-WP-TotalPages']
    return int(pages_count)

def read_wordpress_post_with_pagination():
    total_pages = get_total_pagecount()
    current_page = 1
    all_page_items_json = []
    while current_page <= total_pages:
        api_url = f"{api_prefix}?page={current_page}&per_page=100"
        page_items = requests.get(api_url)
        page_items_json = page_items.json()
        all_page_items_json.extend(page_items_json)
        current_page = current_page + 1
    return all_page_items_json

"""
Transform API JSON to Dataframe
"""
def transform_to_dataframe(api_call):
    api_data = pd.DataFrame(api_call)
    api_data = api_data[['id', 'author', 'date', 'benefit', 'experience', 'technology', 'acf']] # Select columns to work with. Add 'wikidata' when ready.
    api_data['title'] = pd.json_normalize(api_data['acf'])['title_of_creative_work']
    api_data['QID'] = pd.json_normalize(api_data['acf'])['wikidata-qid']
    # This should be cleaner...
    api_data['bene_del'] = pd.json_normalize(api_data['benefit'])
    api_data['benefit'] = pd.json_normalize(api_data['bene_del'])['name']
    api_data['exp_del'] = pd.json_normalize(api_data['experience'])
    api_data['experience'] = pd.json_normalize(api_data['exp_del'])['name']
    api_data['tech_del'] = pd.json_normalize(api_data['technology'])
    api_data['technology'] = pd.json_normalize(api_data['tech_del'])['name']
    del api_data['acf'], api_data['bene_del'], api_data['exp_del'], api_data['tech_del']

    # Convert date of experience to Y-m-d
    api_data['date'] = api_data['date'].str.replace('(\d{4}-\d{2}-\d{2}).*', '\\1', regex = True)
    api_data['date'] = pd.to_datetime(api_data['date'])


    return api_data

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.3 µs


## Write WonderCat API Results to File

In [4]:
%%time

# Call Data from WordPress API
wp_call = read_wordpress_post_with_pagination()

# Reshape wp_call (json) as dataframe.
data = transform_to_dataframe(wp_call)

# Write to file.
data.to_csv("wonderCat_data.tsv", sep = "\t", index = False)

data.head()

CPU times: user 1.75 s, sys: 364 ms, total: 2.12 s
Wall time: 10.3 s


,id,author,date,benefit,experience,technology,title,QID
0,848,6,2025-04-16,Creativity,Wonder,Poetic Language,A Room of One's Own by Virginia Woolf,Q1204366
1,847,6,2025-04-16,Creative Breakthrough,Relearning,Poetic History,A Room of One's Own,Q1204366
2,846,6,2025-04-16,Connection,Superiority,Insinuation,A Room of One's Own,Q1204366
3,845,6,2025-04-16,Resilience,Identification,Soliloquy,A Room of One's Own,Q1204366
4,777,6,2025-03-11,Gratitude,Identification,Stream of Consciousness,To the Lighthouse,Q478016


## WikiData Functions

In [5]:
%%time

# Gather all QID's from dataframe.
def get_QIDS(df):
    # Gather QIDS and validate with regular expression.
    QIDS = df['QID'].unique()
    regex = re.compile('Q\d+')
    QIDS = [s for s in QIDS if regex.match(s)]

    # Append 'wd:' prefix for sparql query.
    QIDS = ' '.join(['wd:' + x for x in QIDS if isinstance(x, str)])

    return QIDS


# Build SPARQL query.
def build_query_call_api(QIDS):
    QIDS = QIDS

    # Build SPARQL Query.
    sparql_query = """
    SELECT DISTINCT
        ?item ?pubDate ?genreLabel
        ?countryOriginLabel ?coordinates

    WHERE {
        VALUES ?item { %s }

        ?item wdt:P31 ?instanceof.
        OPTIONAL {?item wdt:P136 ?genre}.
        OPTIONAL {?item wdt:P577 ?pubDate}.
        ?item wdt:P495 ?countryOrigin .
        ?countryOrigin wdt:P625 ?coordinates.
    
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en,en". }
    }
    """ % (QIDS)

    # Call API
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    res = requests.get(url, params={'query': sparql_query, 'format': 'json'}).json()

    return res

# Create dataframe from API results.
def api_to_dataframe(res):
    wiki_df =[]

    # Loop through WikiQuery Results.
    for i in res['results']['bindings']:
        # Build empty dictionary.
        wiki_item = {}
        # Loop through each item's keys.
        for k in i.keys():
            # Append values to wiki_item
            wiki_item[k] = i[k]['value']

        # Once item's keys looped, append new dictionary to list for dataframe.
        wiki_df.append(wiki_item)

    wiki_df = pd.DataFrame(wiki_df)

    # Clean up item/QID field.
    wiki_df['item'] = wiki_df['item'].str.replace('.*/(Q\d+)', '\\1', regex = True)
    wiki_df = wiki_df.rename(columns = {'item':'QID'})

    # Clean up date field. Currently returning only year due to some dates being "out of bounds" (too old).
    wiki_df['pubDate'] = wiki_df['pubDate'].str.replace('(\d{4}-\d{2}-\d{2}).*', '\\1', regex = True)
    wiki_df['pubDate'] = pd.to_datetime(wiki_df['pubDate'], errors = 'coerce')

    # Create Longitude and Latitude columns.
    reg_pattern = 'Point\(([-]?\d+\.?\d+)\s([-]?\d+\.?\d+)\)'
    wiki_df['long'] = wiki_df['coordinates'].str.replace(reg_pattern, '\\1', regex = True)
    wiki_df['lat'] = wiki_df['coordinates'].str.replace(reg_pattern, '\\2', regex = True)

    # # Convert rows of genres into single value (list)
    # wiki_df = wiki_df.groupby(['QID', 'long', 'lat'], as_index=False) \
    #     .agg({'genreLabel': lambda x: x.tolist(), 'pubDate': lambda x: x.tolist()})

    return wiki_df

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 8.82 µs


In [6]:
%%time

# Get QIDS.
qids = get_QIDS(data)

# Call Wikidata API.
api_results = build_query_call_api(qids)

# Convert API data to dataframe.
wikidata = api_to_dataframe(api_results)

# Merge with WonderCat dataframe.
wikidata = data[['QID', 'title']].merge(wikidata, how = 'inner', on = 'QID')

# Save dataframe as .tsv
wikidata.to_csv("wikidata.tsv", sep = "\t", index = False)

# See if columns that have lists are recognized.
print (wikidata.map(lambda x: isinstance(x, list)).all())

wikidata.head()

QID                   False
title                 False
coordinates           False
genreLabel            False
countryOriginLabel    False
pubDate               False
long                  False
lat                   False
dtype: bool
CPU times: user 33.9 ms, sys: 5.24 ms, total: 39.1 ms
Wall time: 424 ms


,QID,title,coordinates,genreLabel,countryOriginLabel,pubDate,long,lat
0,Q1204366,A Room of One's Own by Virginia Woolf,Point(-1.0 53.0),essay,England,1929-09-01,-1.0,53.0
1,Q1204366,A Room of One's Own,Point(-1.0 53.0),essay,England,1929-09-01,-1.0,53.0
2,Q1204366,A Room of One's Own,Point(-1.0 53.0),essay,England,1929-09-01,-1.0,53.0
3,Q1204366,A Room of One's Own,Point(-1.0 53.0),essay,England,1929-09-01,-1.0,53.0
4,Q478016,To the Lighthouse,Point(-2.0 54.6),modernist literature,United Kingdom,1927-01-01,-2.0,54.6


## Create Network Data with Arguments

In [ ]:
%%time

def create_nodes_and_links(dataframe, column1, column2):
    # Create link/edge pairs.
    title_tech = dataframe[['title', 'technology']]
    title_tech.rename(columns = {'title': 'from', 'technology': 'to'}, inplace = True)

    # Clean pairs of whitespace.
    links['from'] = links['from'].str.replace('\\w', '')
    links['to'] = links['to'].str.replace('\\w', '')

    # Create link/edge weights.
    links = links.groupby(['from', 'to']).size().to_frame(name = 'weight').reset_index()

    # Create nodes from links and rename column name.
    titles = dataframe[['title']]
    titles.rename(columns = {'title': 'label'}, inplace = True)
    titles['category'] = 'title'

    technologies = dataframe[['technology']]
    technologies.rename(columns = {'technology': 'label'}, inplace = True)
    technologies['category'] = 'technology'

    experiences = dataframe[['experience']]
    experiences.rename(columns = {'experience': 'label'}, inplace = True)
    experiences['category'] = 'experience'

    users = dataframe[["author"]]
    users.rename(columns = {'author': 'label'}, inplace = True)
    users['category'] = 'user'

    # Concatenate nodes.
    nodes = pd.concat([titles, technologies, experiences, users]) # users

    # Create node "size" from frequency.
    nodes = nodes.groupby(['label', 'category']).size().to_frame(name = 'size').reset_index()

    # Remove duplicates from nodes.
    nodes.drop_duplicates(inplace = True)

    # Create node "id's."
    nodes['id'] = nodes.index

    # Replace link's 'labels' with node id's.
    label_id_map = pd.Series(nodes['id'].values, index = nodes['label']).to_dict()
    links = links.replace({'from': label_id_map})
    links = links.replace({'to': label_id_map})

    return (links, nodes)

# Create links and nodes.
links, nodes = create_nodes_and_links(data)

# Save data.
links.to_csv("../main/links.tsv", sep = "\t", index = False)
nodes.to_csv("../main/nodes.tsv", sep = "\t", index = False)


## Create Data for Network Graph

In [6]:
%%time

def create_nodes_and_links(dataframe):
    # Create link/edge pairs.
    title_tech = dataframe[['title', 'technology']]
    title_tech.rename(columns = {'title': 'from', 'technology': 'to'}, inplace = True)

    tech_exp = dataframe[['technology', 'experience']]
    tech_exp.rename(columns = {'technology': 'from', 'experience': 'to'}, inplace = True)

    exp_user = dataframe[['experience', 'author']]
    exp_user.rename(columns = {'experience': 'from', 'author': 'to'}, inplace = True)

    # Join pairs.
    links = pd.concat([title_tech, tech_exp, exp_user]) 

    # Clean pairs of whitespace.
    links['from'] = links['from'].str.replace('\\w', '')
    links['to'] = links['to'].str.replace('\\w', '')

    # Create link/edge weights.
    links = links.groupby(['from', 'to']).size().to_frame(name = 'weight').reset_index()

    # Create nodes from links and rename column name.
    titles = dataframe[['title']]
    titles.rename(columns = {'title': 'label'}, inplace = True)
    titles['category'] = 'title'

    technologies = dataframe[['technology']]
    technologies.rename(columns = {'technology': 'label'}, inplace = True)
    technologies['category'] = 'technology'

    experiences = dataframe[['experience']]
    experiences.rename(columns = {'experience': 'label'}, inplace = True)
    experiences['category'] = 'experience'

    users = dataframe[["author"]]
    users.rename(columns = {'author': 'label'}, inplace = True)
    users['category'] = 'user'

    # Concatenate nodes.
    nodes = pd.concat([titles, technologies, experiences, users]) # users

    # Create node "size" from frequency.
    nodes = nodes.groupby(['label', 'category']).size().to_frame(name = 'size').reset_index()

    # Remove duplicates from nodes.
    nodes.drop_duplicates(inplace = True)

    # Create node "id's."
    nodes['id'] = nodes.index

    # Replace link's 'labels' with node id's.
    label_id_map = pd.Series(nodes['id'].values, index = nodes['label']).to_dict()
    links = links.replace({'from': label_id_map})
    links = links.replace({'to': label_id_map})

    return (links, nodes)

# Create links and nodes.
links, nodes = create_nodes_and_links(data)

# Save data.
links.to_csv("../main/links.tsv", sep = "\t", index = False)
nodes.to_csv("../main/nodes.tsv", sep = "\t", index = False)


CPU times: user 123 ms, sys: 6.92 ms, total: 130 ms
Wall time: 158 ms
